In [1]:
%pip install llama-index-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 3.8 MB/s eta 0:00:00


In [2]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 11.3 MB/s eta 0:00:00


In [3]:
import os
import openai

In [4]:
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from IPython.display import Markdown, display

In [6]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

In [7]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [8]:
# create Requirement SQL table
table_name = "requirement_stats"
requirement_stats_table = Table(
    table_name,
    metadata_obj,
    Column("JobID", String(16), primary_key=True),
    Column("Location", String(16), nullable=False),
    Column("CandidatePoolSize", Integer),
    Column("Filled", Integer),
)
metadata_obj.create_all(engine)

In [9]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [10]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [11]:
sql_database = SQLDatabase(engine, include_tables=["requirement_stats"])

In [12]:
sql_database = SQLDatabase(engine, include_tables=["requirement_stats"])
from sqlalchemy import insert

rows = [
    {"JobID": 201, "Location": "New York,NY", "CandidatePoolSize": 25, "Filled": 1},
    {"JobID": 202, "Location": "Austin,TX", "CandidatePoolSize": 15, "Filled": 1},
    {"JobID": 203, "Location": "Miami,FLY", "CandidatePoolSize": 10, "Filled": 0},
    {"JobID": 204, "Location": "Detroit,MI", "CandidatePoolSize": 30, "Filled": 1},
    {"JobID": 205, "Location": "San Francisco,CA", "CandidatePoolSize": 5, "Filled": 0},
    {"JobID": 206, "Location": "Seattle,WA", "CandidatePoolSize": 12, "Filled": 1},
    {"JobID": 207, "Location": "Denver,CO", "CandidatePoolSize": 35, "Filled": 1},
    {"JobID": 208, "Location": "Boston,MA", "CandidatePoolSize": 20, "Filled": 1},
    {"JobID": 209, "Location": "Chicago,IL", "CandidatePoolSize": 8, "Filled": 0},
    {"JobID": 210, "Location": "New York,NY", "CandidatePoolSize": 4, "Filled": 0},
]
for row in rows:
    stmt = insert(requirement_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [13]:
stmt = select(
    requirement_stats_table.c.JobID,
    requirement_stats_table.c.Location,
    requirement_stats_table.c.CandidatePoolSize,
    requirement_stats_table.c.Filled,
).select_from(requirement_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('201', 'New York,NY', 25, 1), ('202', 'Austin,TX', 15, 1), ('203', 'Miami,FLY', 10, 0), ('204', 'Detroit,MI', 30, 1), ('205', 'San Francisco,CA', 5, 0), ('206', 'Seattle,WA', 12, 1), ('207', 'Denver,CO', 35, 1), ('208', 'Boston,MA', 20, 1), ('209', 'Chicago,IL', 8, 0), ('210', 'New York,NY', 4, 0)]


In [14]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT JobID from requirement_stats"))
    for row in rows:
        print(row)

('201',)
('202',)
('203',)
('204',)
('205',)
('206',)
('207',)
('208',)
('209',)
('210',)


In [15]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["requirement_stats"], llm=llm
)
query_str = "Which Location has the highest CandidatePoolsize?"
response = query_engine.query(query_str)

In [16]:
display(Markdown(f"<b>{response}</b>"))

<b>The location with the highest CandidatePoolSize is Denver, CO with a pool size of 35 candidates.</b>

In [17]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="requirement_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [18]:
response = query_engine.query("Which Location has the highest CandidatePoolsize?")
display(Markdown(f"<b>{response}</b>"))

<b>The location with the highest CandidatePoolSize is Denver, CO with a pool size of 35 candidates.</b>

In [19]:
# manually set context text
requirement_stats_text = (
    "This table gives information regarding the CandidatePoolSize and Location of a"
    " given JobID.\nThe user will query with codewords, where 'foo' corresponds"
    " to CandidatePoolSize and 'bar'corresponds to Location."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="requirement_stats", context_str=requirement_stats_text))
]

In [20]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["requirement_stats"], return_raw=True
)

In [21]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 Locations (along with their CandidatePoolSize) with the highest CandidatePoolSize."
)

In [22]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

**Node ID:** 9849c104-8615-4c9d-bff0-a3973a5dfd6a<br>**Similarity:** None<br>**Text:** [('Denver,CO', 35), ('Detroit,MI', 30), ('New York,NY', 25), ('Boston,MA', 20), ('Austin,TX', 15)]<br>

In [23]:
# default retrieval (return_raw=False)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["requirement_stats"], return_raw=False
)

In [24]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 Locations (along with their CandidatePoolSize) with the highest CandidatePoolSize."
)

In [25]:
# NOTE: all the content is in the metadata
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** da92c4cb-d0df-494a-a073-d96cf9ee4a06<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'Location': 'Denver,CO', 'CandidatePoolSize': 35}<br>

**Node ID:** 4c485ec4-3fe8-494c-ad6f-309ce39157e5<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'Location': 'Detroit,MI', 'CandidatePoolSize': 30}<br>

**Node ID:** d8dee835-8a25-4526-8a01-f7052abb3309<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'Location': 'New York,NY', 'CandidatePoolSize': 25}<br>

**Node ID:** 448aa122-32c3-4dec-ad11-1155fe5e4880<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'Location': 'Boston,MA', 'CandidatePoolSize': 20}<br>

**Node ID:** 6272e9e4-3b17-4172-9598-319190201539<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'Location': 'Austin,TX', 'CandidatePoolSize': 15}<br>

In [26]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [27]:
response = query_engine.query(
    "Return the top 5 Locations (along with their CandidatePoolSize) with the highest CandidatePoolSize."
)

In [28]:
print(str(response))

Detroit,MI: 30
Denver,CO: 35
New York,NY: 25
Boston,MA: 20
Austin,TX: 15
